In [ ]:
%matplotlib inline

import sys
sys.path.insert(0, '/home/Matthew.Thomas/python_code/python_pkgs/MOC_funcs')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs
from numba import njit
import sectionate
from glob import glob
import momlevel
from xhistogram.xarray import histogram
import MOC_funcs
from cmip_basins.basins import generate_basin_codes
import julian

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import dask
# from dask.distributed import Client
# from dask_jobqueue import SLURMCluster
# dask.config.set(**{'array.slicing.split_large_chunks': False})

# portdash = 18224
# # portdash = 18234

# cluster = SLURMCluster(
#     queue="batch",
#     cores=8,
#     processes=4,
#     project="gfdl_o",
#     memory="48GB",
#     walltime="16:00:00",
#     local_directory="$TMPDIR",
#     death_timeout=120,
#     scheduler_options={"dashboard_address":f":{portdash}"},
# )

# client = Client(cluster)
# client

In [ ]:
# cluster.scale(jobs=10)

In [ ]:
run_dict = {
                'odiv209' : {
                             'dir_vars' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/",
                             'dir_eddy_vars' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z_d2/ts/annual/10yr/",
                             'dir_grid' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'dir_grid_eddy' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z_d2/ocean_annual_z_d2.static.nc",
                             'file_str_identifier' : "*",
                             'lat_range' : [26.3], 
                             'basin_list': [2],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i",
                             'color_identifier' : 'b',
                             },
                'odiv230' : {
                             'dir_vars' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/",
                             'dir_eddy_vars' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/",
                             'dir_grid' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'dir_grid_eddy' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*",
                             'lat_range' : [26.3],
                             'basin_list': [2],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i",
                             'color_identifier' : 'yellow',
                             },
                'odiv1' :   {
                             'dir_vars' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ts/annual/5yr/",
                             'dir_eddy_vars' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ts/annual/5yr/",
                             'dir_grid' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'dir_grid_eddy' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*.0[0-2]*",
                             'lat_range' : [26.3],
                             'basin_list': [2],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i",
                             'color_identifier' : 'pink',
                             },
        }

In [ ]:
%%time
### AMOCz in CM4hires

dmgetout_flag = False
integrate_downwards_flag=False
zarr_dir='/xtmp/Matthew.Thomas/work'

ComputeDiag_dict = {}
if dmgetout_flag: print(f'run the following dmget commands:')
for keys in run_dict:
    ds_z_online, grid_z_online, moc_z_online, vmo_zonmean_online = MOC_funcs.MOC_basin_latrange_online(dir_vars=run_dict[keys]['dir_vars'],dir_grid=run_dict[keys]['dir_grid'],file_str_identifier=run_dict[keys]['file_str_identifier'],lat_range=run_dict[keys]['lat_range'],basin_list=run_dict[keys]['basin_list'],z_layer_var=run_dict[keys]['z_layer_var'],z_inter_var=run_dict[keys]['z_inter_var'],decode_times_flag=True,dmgetout=dmgetout_flag,zarr_dir=zarr_dir)
    ds_z_heat, grid_z_heat, moc_z_heat, vmo_zonmean_heat = MOC_funcs.MOC_basin_latrange_online(dir_vars=run_dict[keys]['dir_eddy_vars'],dir_grid=run_dict[keys]['dir_grid_eddy'],file_str_identifier=run_dict[keys]['file_str_identifier'],lat_range=run_dict[keys]['lat_range'],basin_list=run_dict[keys]['basin_list'],z_layer_var=run_dict[keys]['z_layer_var'],z_inter_var=run_dict[keys]['z_inter_var'],decode_times_flag=True,v_transport_var="T_ady",dmgetout=dmgetout_flag,zarr_dir=zarr_dir)
    ds_z_eddy, grid_eddy, moc_z_eddy, vmo_z_zonmean_eddy = MOC_funcs.MOC_basin_latrange_online(dir_vars=run_dict[keys]['dir_eddy_vars'],dir_grid=run_dict[keys]['dir_grid_eddy'],file_str_identifier=run_dict[keys]['file_str_identifier'],lat_range=run_dict[keys]['lat_range'],basin_list=run_dict[keys]['basin_list'],z_layer_var=run_dict[keys]['z_layer_var'],z_inter_var=run_dict[keys]['z_inter_var'],decode_times_flag=True,v_transport_var="vhml",dmgetout=dmgetout_flag,zarr_dir=zarr_dir)
    
    if len(ds_z_online)>0:
        MHT_online=ds_z_heat.sum(dim=['xh','z_l'])
        
        ds_z_eddy['time']=ds_z_online['time']
        polarity_multiplier=1 if integrate_downwards_flag else -1
        moc_z_resid=polarity_multiplier*((ds_z_online.vmo.sum('xh').sortby('z_l', ascending=integrate_downwards_flag).cumsum('z_l'))-(ds_z_eddy.vhml.rename('vmo').sum('xh').sortby('z_l', ascending=integrate_downwards_flag).cumsum('z_l')))

        ComputeDiag_dict[keys]={
                                'MHT_online' : MHT_online, 'moc_z_resid' : moc_z_resid,
                                'ds_z_online' : ds_z_online, 'grid_z_online' : grid_z_online, 'moc_z_online' : moc_z_online, 'vmo_zonmean_online' : vmo_zonmean_online,
                                'ds_z_eddy' : ds_z_eddy, 'moc_z_eddy' : moc_z_eddy, 'vmo_z_zonmean_eddy' : vmo_z_zonmean_eddy,
                                }

In [ ]:
### Now load RAPID observations

In [ ]:
### Load RAPID volume transport
RAPID_streamfunction=xr.open_dataset('/home/Matthew.Thomas/archive/data/RAPID/moc_vertical.nc',decode_times=True)
RAPID_transports=xr.open_dataset('/home/Matthew.Thomas/archive/data/RAPID/moc_transports.nc',decode_times=True)

In [ ]:
### Load RAPID heat transport
#See this page for details for the RAPID-MOCHA variable names: https://docs.google.com/document/d/1TEbVP5dd9dot9_IWUlRc0ipg-eAdzZ1_/edit
ds_RAPID_MOCHA=xr.open_dataset('/home/Matthew.Thomas/archive/data/RAPID/mocha_mht_data_ERA5_v2018_2.nc',decode_times=True)
dates = [julian.from_jd(x) for x in ds_RAPID_MOCHA.julian_day]  # list-comprehension for converting the julian days into calendar dates
ds_RAPID_MOCHA = ds_RAPID_MOCHA.assign_coords(time=dates) # Assign these calendar dates to a new coordinate called 'time'. 

In [ ]:
# Resample the twice-daily data into monthly data (Note that .resample is a .groupby operation that is specific to time)
RAPID_transports_annual=RAPID_transports.resample(time="Y").mean().moc_mar_hc10
ds_RAPID_MOCHA_annual=ds_RAPID_MOCHA.resample(time="Y").mean().Q_ot

In [ ]:
%%time
smoothing_level=10   #in years
load_data_flag=1
fig = plt.figure(figsize=(24,5))
start_dates=[]
end_dates=[]
for keys in run_dict:
    
    if load_data_flag:
        ComputeDiag_dict[keys]['moc_z_resid']=ComputeDiag_dict[keys]['moc_z_resid'].load()
        ComputeDiag_dict[keys]['MHT_online']=ComputeDiag_dict[keys]['MHT_online'].load()
    
    plt.subplot(1,3,1)
    moc_z_resid_sf=(ComputeDiag_dict[keys]['moc_z_resid'].mean('time')/1030/1e6).plot(y='z_l',ylim=[6000,0],label=keys,color=run_dict[keys]['color_identifier'])
    plt.subplot(1,3,2)
    moc_z_resid_ts=(ComputeDiag_dict[keys]['moc_z_resid'].where(ComputeDiag_dict[keys]['moc_z_resid'].z_l>500).max(dim='z_l')/1030/1e6)
    moc_z_resid_ts.plot(color=run_dict[keys]['color_identifier'],alpha=0.7,linewidth=.5)
    moc_z_resid_ts.rolling(time=smoothing_level,center=True).mean().plot(label=keys,color=run_dict[keys]['color_identifier'])
    plt.subplot(1,3,3)
    mht_z_ts=(ComputeDiag_dict[keys]['MHT_online'].T_ady/1e15)
    mht_z_ts.plot(color=run_dict[keys]['color_identifier'],alpha=0.7,linewidth=.5)
    mht_z_ts.rolling(time=smoothing_level,center=True).mean().plot(label=keys,color=run_dict[keys]['color_identifier'])
    start_dates.append(ComputeDiag_dict[keys]['moc_z_resid'].time.values[0])
    end_dates.append(ComputeDiag_dict[keys]['moc_z_resid'].time.values[-1])
plt.subplot(1,3,1)
RAPID_streamfunction.stream_function_mar.mean('time').plot(label='obs',y='depth',ylim=[6000,0],color='k')
plt.legend()
plt.subplot(1,3,2)
# plt.fill_between([moc_z_resid.time.values[0],moc_z_resid.time.values[-1]],RAPID_transports_annual.mean()-RAPID_transports_annual.std(),RAPID_transports_annual.mean()+RAPID_transports_annual.std(), alpha=0.2, label='obs (mean+/-std')
plt.fill_between([np.min(start_dates),np.max(end_dates)],RAPID_transports_annual.mean()-RAPID_transports_annual.std(),RAPID_transports_annual.mean()+RAPID_transports_annual.std(), alpha=0.2, label='annual obs (mean+/-std)')
plt.xlabel('time (years)'), plt.ylabel('Sv')
plt.title('annual RAPID MOC, '+str(smoothing_level)+'-year smoothing')
plt.subplot(1,3,3)
plt.fill_between([np.min(start_dates),np.max(end_dates)],(ds_RAPID_MOCHA_annual.mean()-ds_RAPID_MOCHA_annual.std())/1e15,(ds_RAPID_MOCHA_annual.mean()+ds_RAPID_MOCHA_annual.std())/1e15, alpha=0.2, label='annual obs (mean+/-std)')
plt.xlabel('time (years)'), plt.ylabel('PW')
plt.title('annual RAPID MHT, '+str(smoothing_level)+'-year smoothing')
plt.legend()

In [ ]:
cluster.close()
client.close()

In [ ]:
# Done! 